In [2]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import os
import re
import csv

### If the files you want are already in the csv-files folder, just skip to step 7

# Step 1: Open File

In [324]:
fileName = input("What is the name of the file (format: SemesterNameYY.html): ")
x = 0
while fileName not in os.listdir("workday-data") and x < 3:
  fileName = input("{} not found. Try again".format(fileName))
  x += 1
if x >= 3 and fileName not in os.listdir("workday-data"):
  raise Exception("File not found")
print("Success! fileName = {}".format(fileName))

Success! fileName = Spring 25.html


In [325]:
file_path = "workday-data/" + fileName

with open(file_path, "r") as file:
    content = file.read()

soup = BeautifulSoup(content, "html.parser")
semester = fileName.replace(".html", "")
# WOVQ WPUQ WHWQ
# data-automation-id= ["promptOption", "menuItem", "compositeSubHeaderOne", ...]
# promptOption: course, schedule, schedule-details, classmode, prqs
# menuItem: schedule, schedule-details, classmode, prqs
# compositeSubHeaderOne: instructor
div_store = {"Spring 25" : "WGVF WEVF",\
             "Fall 24" : "WJVF WHVF",
             "Spring 24" : "WJVF WHVF",
             "Fall 23" : "WJVF WHVF",
             "Spring 23" : "WJVF WHVF",
             "Fall 22" : "WJVF WHVF"}
# found these manually, may be a way to find them automatically?
# seemingly, either Workday changed the code for last spring specifically, 
# or archived datacodes are stored differently as opposed to recent datacodes
course_classes = soup.find_all(attrs={"class": div_store[semester]})

# Step 2: Parse File for Courses

In [326]:
# Each course is stored under <div class="WGVF WEVF"
# this class changes per webpage, but here is wgvf wevf
# WJVF WHVF

id_lod = []
for course_data in tqdm(course_classes):
  # grab course from this div
  all_ids = course_data.find_all(attrs={"id" : lambda x: x and "gwt-uid-" in x})
  x = 0
  id_dict = {}
  for element in all_ids:
    # course should be lowest id
    # remove the prompt option:
    uid = int(element.attrs["id"].replace("promptOption-", "").replace("gwt-uid-",""))

    if element.text not in id_dict.values():
      id_dict[uid] = element.text

  id_lod.append(id_dict)


# PRQS and other section details are written somewhere else, under same div

100%|██████████| 1121/1121 [00:00<00:00, 5704.50it/s]


In [327]:
# # use this to test individual indexes
# # for item in id_lod:
# #   print(id_lod.index(item), len(item))
# id_lod[895]

# Step 3: Sort by UID

In [328]:
id_df = pd.DataFrame([], index=["gwt-uid", "info", "courseIdx"])
for idx in range(len(id_lod)):
  # print(idx)
  dict_lst = [id_lod[idx].keys(), id_lod[idx].values(), [idx]*len(id_lod[idx])]
  id_df = pd.concat([id_df, pd.DataFrame(dict_lst, index=["gwt-uid", "info", "courseIdx"])], axis=1)
id_df = id_df.T
id_df
# id_lod[0]

,gwt-uid,info,courseIdx
0,24,ACC 120-01 - Principles of Financial and Manag...,0
1,26,Closed | Mark Bettner | In-Person | ...,0
2,25,HIP-202 | TR | 8:30 AM - 9:45 AM,0
3,3026,Lecture,0
4,3027,PRQ - (No credit for ACC 111 or ACC 301) AND (...,0
...,...,...,...
3,7024,"GER - CORE :: TA (Critical, Analytical Interpr...",1120
4,7025,"GER - Global Awareness :: IEJ (Identities, Equ...",1120
5,7026,"IDS :: Womenâ€™s, Gender, and Sexuality Studies",1120
6,7027,Lecture,1120


In [329]:
# id_lod[0]

In [330]:
id_df = id_df.reset_index(drop=True).sort_values(by="gwt-uid", ascending=True)

In [331]:
num_classes = id_df["courseIdx"].max() + 1
num_classes

1121

# Step 4: Convert *Info* to usable data (Course: Instructor: Section)



In [2]:
# min = id_df["gwt-uid"].min()
min_cond = id_df["courseIdx"] == 1
# the idea is that the minimum value in the next course is always higher than smaller values
# in the first course, but it is smaller than the larger values in the first course
threshold = id_df[min_cond]["gwt-uid"].min()
upper_limit = id_df["gwt-uid"].max()

for id in id_df[id_df["courseIdx"]==0]["gwt-uid"]:
  if id > threshold:
    if id < upper_limit:
      upper_limit = id
upper_limit

NameError: name 'id_df' is not defined

In [333]:
# verify the upper limit is accurate manually
id_df[id_df["courseIdx"]==0]

,gwt-uid,info,courseIdx
0,24,ACC 120-01 - Principles of Financial and Manag...,0
2,25,HIP-202 | TR | 8:30 AM - 9:45 AM,0
1,26,Closed | Mark Bettner | In-Person | ...,0
3,3026,Lecture,0
4,3027,PRQ - (No credit for ACC 111 or ACC 301) AND (...,0


In [334]:
course_cond = id_df["courseIdx"] == 0
lower_cond = id_df["gwt-uid"] < upper_limit
for item in id_df[course_cond].values:
  print(item)

[24 'ACC 120-01 - Principles of Financial and Managerial Accounting' 0]
[25 'HIP-202 | TR | 8:30 AM - 9:45 AM' 0]
[26 'Closed   |   Mark Bettner   |   In-Person   |   25/24' 0]
[3026 'Lecture' 0]
[3027
 'PRQ - (No credit for ACC 111 or ACC 301) AND (First-Year or Sophomore Class Standing)'
 0]


In [335]:
# with all these things sorted by gwt-uid, let's assign courses, schedule, instructor
# class_len = len(id_lod)
d = []
lower_cond  = id_df["gwt-uid"] < upper_limit
try:
  for course_idx in range(num_classes):
    course_cond = id_df["courseIdx"] == course_idx
    course_values = id_df[lower_cond & course_cond]

    course_scale = len(course_values)
    if course_scale == 3:
      course, schedule, instructor = [value for value in course_values["info"].values]
    elif course_scale == 2:
      course, instructor = [value for value in course_values["info"].values]
      schedule = "empty | empty | empty"

    # deconstructing course
    course_result = course.split("-")
    course_result = [x.strip() for x in course_result]
    if len(course_result) <= 3:
      courseCode, courseSection, courseTitle = course_result
    elif len(course_result) > 3:
      # in case the title has dashes in it
      courseCode, courseSection, courseTitle  = (course_result[0], course_result[1], "-".join((course_result[2:])))

    # deconstructing schedule (if it exists)
    # the [-1] checks if there is a date attached (some courses do that)
    schedule_result = schedule.split(" | ")
    if len(schedule_result) == 3 and "/"  not in schedule_result[-1]:
      roomId, days, time                      = schedule_result
    elif len(schedule_result) == 3 and "/" in schedule_result[-1]:
      days, time                              = schedule_result[0], schedule_result[1]
      roomId = "empty"

    elif len(schedule_result) == 4 and "/" in schedule_result[-1]:
      roomId, days, time                      = (schedule_result[0], schedule_result[1], schedule_result[2])

    # deconstructing instructor
    instructor_result = instructor.split(" | ")
    if len(instructor_result) == 4:
      cStatus, insName, cMode, cFill          = [x.strip() for x in instructor_result]
    else:
      cStatus, cMode, cFill                   = [x.strip() for x in instructor_result]
      insName = "empty"

    # deal with this later
    detail_values = id_df[course_cond & ~lower_cond]
    other = detail_values["info"].values
    tags, prqs, cluster, others = ["empty"], ["empty"], ["empty"], ["empty"]
    format = "empty"

    for detail in other:
      if detail in ["Lecture", "Laboratory", "Seminar", "Studio", "Independent Study",\
                    "Experiential", "Discussion", "Internship", "Combination"]:
        format = detail

      # specifically in Fall 23, they formatted "AR - GER" for FYW prereqs
      elif detail.startswith("PRQ") or detail.startswith("AR - GER"):
        prqs.remove("empty") if "empty" in prqs else None
        if detail.startswith("PRQ"):
          prqs.append(detail.replace("PRQ - ", ""))
        else:
          prqs.append(detail.replace("AR - GER - ", ""))

      elif detail.startswith("GER") or detail.startswith("IDS") or \
      detail.startswith("Special Course") or detail.startswith("FYW topics"):
        tags.remove("empty") if "empty" in tags else None
        tags.append(detail)

      # clusters will be dealt with later


      else:
        others.remove("empty") if "empty" in others else None
        others.append(detail)

    d.append([courseCode, courseSection, courseTitle, [roomId], [days], [time],\
              cStatus, insName, cMode, cFill, tags, prqs, cluster, [format], others])

except Exception as e:
  print(e)
  print(course_idx)
d;

In [336]:
# in the dictionary keys, if the value is less than the length of id_lod (i.e. # classes), check if its a course, schedule, or instructor
# otherwise, check if its a schedule-details, courseTags, format, prqs, or course cluster
courses_df = pd.DataFrame(d, columns=["CourseCode", "Section", "CourseTitle", "RoomId", "Days", "Time", "Status", "InstructorName", "Mode", "Fill", "CourseTags", "Prerequisites", "CourseCluster", "InstructionalFormat", "Other"])
course = courses_df["CourseCode"] == "BIO 111"
# sect = courses_df["Section"] == "01"
# courses_df[course & sect]["Other"].values
courses_df.loc[course]

,CourseCode,Section,CourseTitle,RoomId,Days,Time,Status,InstructorName,Mode,Fill,CourseTags,Prerequisites,CourseCluster,InstructionalFormat,Other
55,BIO 111,01,Foundations of Biology,[PYR-028],[TR],[8:30 AM - 9:45 AM],Open,Wade Worthen,In-Person,23/28,[GER - CORE :: NWL - Natural World with Lab (m...,[empty],[empty],[Lecture],[BIO 111-01L - Foundations of Biology]
56,BIO 111,01L,Foundations of Biology,[PYR-142],[T],[2:30 PM - 5:20 PM],Open,Wade Worthen,In-Person,23/28,[GER - CORE :: NWL - Natural World with Lab (m...,[empty],[empty],[Laboratory],[BIO 111-01 - Foundations of Biology]
57,BIO 111,02,Foundations of Biology,[PYR-026],[MWF],[11:30 AM - 12:20 PM],Open,Grace Freundlich,In-Person,24/28,[GER - CORE :: NWL - Natural World with Lab (m...,[empty],[empty],[Lecture],[BIO 111-02L - Foundations of Biology]
58,BIO 111,02L,Foundations of Biology,[PYR-142],[W],[2:30 PM - 5:20 PM],Open,Grace Freundlich,In-Person,24/28,[GER - CORE :: NWL - Natural World with Lab (m...,[empty],[empty],[Laboratory],[BIO 111-02 - Foundations of Biology]
59,BIO 111,03,Foundations of Biology,[PYR-026],[TR],[10:00 AM - 11:15 AM],Open,David Hollis,In-Person,16/28,[GER - CORE :: NWL - Natural World with Lab (m...,[empty],[empty],[Lecture],[BIO 111-03L - Foundations of Biology]
60,BIO 111,03L,Foundations of Biology,[PYR-142],[R],[2:30 PM - 5:20 PM],Open,David Hollis,In-Person,16/28,[GER - CORE :: NWL - Natural World with Lab (m...,[empty],[empty],[Laboratory],[BIO 111-03 - Foundations of Biology]


In [337]:
# # use this to test individual indexes
# course_idx = int(input("# "))
# courses_df[course_idx:course_idx+1]

## Step 4.5: Create Prefixes, Locs to fill *CourseCluster*

In [338]:
prefix_lst = sorted(set(courses_df["CourseCode"].apply(lambda x: x.split(" ")[0])))
print(prefix_lst)

locations = sorted(set(courses_df["RoomId"].apply(lambda x: x[0].split("-")[0] if x != "empty" else x)))
locations.remove("empty")
print(locations)

['ACC', 'AFS', 'ANT', 'ART', 'AST', 'BIO', 'BUS', 'CHM', 'CHN', 'CLS', 'COM', 'CSC', 'DAN', 'ECN', 'EDU', 'EES', 'ENG', 'EST', 'FRN', 'FST', 'FYW', 'GRK', 'GRM', 'HSC', 'HST', 'HUM', 'IGD', 'JPN', 'LAS', 'LNG', 'LTN', 'MEM', 'MES', 'MLL', 'MSL', 'MTH', 'MUS', 'NSC', 'PHL', 'PHY', 'POL', 'PSY', 'PTH', 'PVS', 'REL', 'SCI', 'SOC', 'SPN', 'SUS', 'THA', 'UNV', 'WGS']
['BCM', 'CHP', 'DMB', 'FUR', 'HIP', 'HMP', 'JHN', 'LIB', 'PAC', 'PYR', 'RAB', 'RLY', 'RNK', 'SHI', 'TNS', 'TPH']


In [339]:
index = 0
cond = courses_df["Other"].apply(lambda x: True if x == ("empty") else False)
# cond = courses_df["CourseCode"]=="ACC 120"
courses_df[~cond]["CourseCluster"]
# courses_df.loc[0, "Other"][0]

0       [empty]
1       [empty]
2       [empty]
3       [empty]
4       [empty]
         ...   
1116    [empty]
1117    [empty]
1118    [empty]
1119    [empty]
1120    [empty]
Name: CourseCluster, Length: 1121, dtype: object

In [340]:
# course = courses_df["CourseCode"] == "PSY 201"
# sect = courses_df["Section"] == "01"
# courses_df[course & sect]["CourseCluster"]

# Step 5: Fill *CourseCluster* from remaining *Other* data

In [341]:
for idx in courses_df.index:
  # other = courses_df.loc[idx, "Other"]
  if courses_df.loc[idx, "Other"] != ["empty"]:
    store = []
    for item in courses_df.loc[idx, "Other"]:

      # courseCluster gets resolved here - only courses get added
      if item.startswith(tuple(prefix_lst)):
        courses_df.loc[idx, "CourseCluster"].remove("empty") if "empty" in courses_df.loc[idx, "CourseCluster"] else courses_df.loc[idx, "CourseCluster"]
        courses_df.loc[idx, "CourseCluster"].append(item)
        store.append(item)

      # some courses have two "schedules", this should resolve that
      elif item.startswith(tuple(locations)):
        # if there is a date there
        if len(item.split(" | ")) > 3:
          split = item.split(" | ")
          roomId, days, time = split[0], split[1], split[2]
        else:
          roomId, days, time = item.split(" | ")
        courses_df.loc[idx, "RoomId"].append(roomId)
        courses_df.loc[idx, "Days"].append(days)
        courses_df.loc[idx, "Time"].append(time)
        store.append(item)

      # but what if there is no roomId, muhahahaha
      elif item.startswith(("M", "T", "W", "R", "F")):
        days, time = item.split(" | ")
        roomId = "empty"
        courses_df.loc[idx, "RoomId"].append(roomId)
        courses_df.loc[idx, "Days"].append(days)
        courses_df.loc[idx, "Time"].append(time)
        store.append(item)

    for item in store:
      courses_df.loc[idx, "Other"].remove(item) if item in courses_df.loc[idx, "Other"] else courses_df.loc[idx, "Other"]
      if len(courses_df.loc[idx, "Other"]) == 0:
        courses_df.loc[idx, "Other"] = [("empty")]


courses_df;
    # print(other)

In [342]:
# ONLY RUN THIS ONCE
courses_df["CourseCluster"] = courses_df["CourseCluster"].apply(lambda x: ", ".join(x) if x != ["empty"] else "empty")
# courses_df["Other"] = courses_df["Other"].apply(lambda x: "\n".join(x)

In [343]:
# course = courses_df["CourseCode"] == "BIO 111"
# sect = courses_df["Section"] == "02"
# courses_df[course & sect]#["CourseCluster"]

In [344]:
# check to make sure these are just courses
for item in courses_df[(courses_df['CourseCluster']!="empty")]["CourseCluster"]:
  print(item)

ANT 306-01L - Human Osteology & Paleopathology
ANT 306-01 - Human Osteology & Paleopathology
BIO 102-01L - Biology and the Environment
BIO 102-01 - Biology and the Environment
BIO 111-01L - Foundations of Biology
BIO 111-01 - Foundations of Biology
BIO 111-02L - Foundations of Biology
BIO 111-02 - Foundations of Biology
BIO 111-03L - Foundations of Biology
BIO 111-03 - Foundations of Biology
BIO 210-01L - Genetics
BIO 210-01 - Genetics
BIO 210-02L - Genetics
BIO 210-02 - Genetics
BIO 210-03L - Genetics
BIO 210-03 - Genetics
BIO 210-04L - Genetics
BIO 210-04 - Genetics
BIO 222-01L - Research and Analysis
BIO 222-01 - Research and Analysis
BIO 222-02L - Research and Analysis
BIO 222-02 - Research and Analysis
BIO 303-01L - Biochemistry of the Cell
BIO 303-01 - Biochemistry of the Cell
BIO 320-01L - Animal Physiology
BIO 320-01 - Animal Physiology
BIO 322-01L - Human Physiology
BIO 322-01 - Human Physiology
BIO 340-01L - Ecology
BIO 340-01 - Ecology
BIO 340-02L - Ecology
BIO 340-02 - Ecol

In [345]:
is_empty = courses_df["Other"].apply(lambda x: False if x == [("empty")] else True)
courses_df[is_empty]

,CourseCode,Section,CourseTitle,RoomId,Days,Time,Status,InstructorName,Mode,Fill,CourseTags,Prerequisites,CourseCluster,InstructionalFormat,Other


In [346]:
large_sched_cond = courses_df["RoomId"].apply(lambda x: False if len(x) == 1 else True)
courses_df[large_sched_cond]

,CourseCode,Section,CourseTitle,RoomId,Days,Time,Status,InstructorName,Mode,Fill,CourseTags,Prerequisites,CourseCluster,InstructionalFormat,Other
75,BIO 320,01,Animal Physiology,"[PYR-143, PYR-143]","[MWF, F]","[9:30 AM - 10:20 AM, 8:30 AM - 9:20 AM]",Open,Dennis Haney,In-Person,15/18,[IDS :: Neuroscience],[BIO 222 Research and Analysis],BIO 320-01L - Animal Physiology,[Lecture],[empty]
414,HSC 102,01,Mindfulness for Stress Reduction and Wellness,"[CHP-GAR, CHP-GAR]","[W, F]","[8:30 AM - 10:20 AM, 8:30 AM - 9:20 AM]",Closed,Scott Murr,In-Person,20/20,[GER - CORE :: MB (Mind and Body)],[Instructor Permission Required],HSC 102-01L - Mindfulness for Stress Reduction...,[Lecture],[empty]


In [347]:
if len(courses_df[is_empty]) == 0:
  courses_df = courses_df.drop(columns=["Other"])
  print("Success")
else:
  print("Uh oh")

Success


In [348]:
courses_df;

In [349]:
# # find index of given course
# courses_idx = int(input("#"))
# courses_df[courses_idx:courses_idx+1][["CourseCode", "Section", "CourseTags", \
#                                        "Prerequisites", "CourseCluster", "InstructionalFormat"]]

# Step 6: Send data to .csv for web-parsing

In [ ]:
# first, make things into slash-delimited strings, not lists - lists complicate js a bit
    # not comma-delimited - eg: some majors have commas in their titles
# RUN ONCE
courses_df = courses_df.apply(lambda x: x.apply(lambda y: "/".join(y) if type(y) == list else y))

In [351]:
# second, filter things we actually need:
courses = courses_df[["CourseCode", "Section", "CourseTitle", "CourseTags", "Prerequisites", "CourseCluster", "InstructionalFormat"]]
courses

,CourseCode,Section,CourseTitle,CourseTags,Prerequisites,CourseCluster,InstructionalFormat
0,ACC 120,01,Principles of Financial and Managerial Accounting,empty,(No credit for ACC 111 or ACC 301) AND (First-...,empty,Lecture
1,ACC 120,02,Principles of Financial and Managerial Accounting,empty,(No credit for ACC 111 or ACC 301) equivalency...,empty,Lecture
2,ACC 120,03,Principles of Financial and Managerial Accounting,empty,(No credit for ACC 111 or ACC 301),empty,Lecture
3,ACC 120,04,Principles of Financial and Managerial Accounting,empty,(No credit for ACC 111 or ACC 301) AND (First-...,empty,Lecture
4,ACC 311,01,Intermediate Accounting I,empty,"(ACC 111 or ACC 120) AND (First-Year, Sophomor...",empty,Lecture
...,...,...,...,...,...,...,...
1116,UNV TSGR,01,Germany-Travel Study,empty,empty,empty,Lecture
1117,UNV TSIT,01,Italy-Travel Study,empty,empty,empty,Lecture
1118,UNV TSLN,01,England-Furman in London,empty,Instructor Permission Required,empty,Experiential
1119,WGS 101,01,Issues in Women's Gender and Sexuality Studies,"GER - Global Awareness :: IEJ (Identities, Equ...",empty,empty,Lecture


In [352]:
# drop any repetitive lines
final_courses = courses.drop_duplicates()

In [353]:
len(final_courses), len(courses)

(1121, 1121)

In [113]:
semester_order = ["Fall 22", "Spring 23", "Fall 23", "Spring 24", "Fall 24", "Spring 25"]
newFileName = fileName.replace(".html", "")
newFileName = str(semester_order.index(newFileName)) + "_" + fileName.replace(" ", "-")
newFileName

'0_Fall-22.html'

In [ ]:
file_path = "csv-files/{}".format(newFileName.replace(".html",".csv"))
final_courses.to_csv(file_path)

# Step 7: Merge all .csv files into one workday-courses.csv file

In [3]:
file_lst = []
for file in os.listdir("csv-files"):
    if file != "programs.csv" and file != "gers.csv":
        file_df = pd.read_csv("csv-files/{}".format(file))
        file_df = file_df.drop(columns=["Unnamed: 0"])
        file_df["Semester"] = file.replace(".csv", "")
        file_lst.append(file_df)

print(len(file_lst[0]), len(file_lst[1]), len(file_lst[2]), len(file_lst[3]), len(file_lst[4]), len(file_lst[5]))
print(sum((\
            len(file_lst[0]),\
            len(file_lst[1]),\
            len(file_lst[2]),\
            len(file_lst[3]),\
            len(file_lst[4]),\
            len(file_lst[5])\
        )))
all_workday_df= pd.concat(file_lst, ignore_index=True).sort_values(by="CourseCode").drop_duplicates(subset=['CourseCode', 'Section',\
                                                                              'CourseTitle', 'CourseTags', 'Prerequisites',\
                                                                              'CourseCluster', 'InstructionalFormat'])\
                                                                                .reset_index().drop(columns=["index"])
print(len(all_workday_df))
all_workday_df[all_workday_df["CourseCode"]=="ANT 101"];

1072 1065 1098 1076 1105 1121
6537
3022


# Step 8: Pull out appropriate Prereqs and Course Tags

In [4]:
list_tags = all_workday_df["CourseTags"].apply(lambda x: x.split("/"))
# list_tags
all_workday_df

,CourseCode,Section,CourseTitle,CourseTags,Prerequisites,CourseCluster,InstructionalFormat,Semester
0,ACC 111,03,Principles of Financial Accounting,empty,empty,empty,Lecture,0_Fall-22
1,ACC 111,04,Principles of Financial Accounting,empty,empty,empty,Lecture,1_Spring-23
2,ACC 111,01,Principles of Financial Accounting,empty,PRQ - Department Chair Permission Required,empty,Lecture,2_Fall-23
3,ACC 111,02,Principles of Financial Accounting,empty,PRQ - Department Chair Permission Required,empty,Lecture,2_Fall-23
4,ACC 111,01,Principles of Financial Accounting,empty,First Year or Sophomore Class Standing,empty,Lecture,1_Spring-23
...,...,...,...,...,...,...,...,...
3017,WGS 240,01,Women in Science,"IDS :: Medicine, Health and Culture/IDS :: Sci...",empty,empty,Lecture,4_Fall-24
3018,WGS 250,01,Introduction to Queer Theory and Sexuality Stu...,"GER - CORE :: TA (Critical, Analytical Interpr...",FYW (First Year Writing Seminar),empty,Lecture,5_Spring-25
3019,WGS 503,02,Supervised Internship,"IDS :: Womenâ€™s, Gender, and Sexuality Studies",Instructor permission required,empty,Independent Study,4_Fall-24
3020,WGS 503,01,Supervised Internship,"IDS :: Womenâ€™s, Gender, and Sexuality Studies",Instructor permission required,empty,Independent Study,1_Spring-23


In [5]:
list_tags = all_workday_df["CourseTags"].apply(lambda x: x.split("/"))
all_lst_df = all_workday_df.copy()
all_lst_df["CourseTags"] = list_tags
# new_series = pd.Series(index=range(len(list_tags)))
new_tags = all_lst_df["CourseTags"]
new_tags
fywless_tags = pd.DataFrame()
idx = 0
for x in new_tags:
    # print(x)
    # new_lst = [string for string in x if not string.startswith("FYW topics")]
    new_lst = [string for string in x if "FYW topics" not in string and "IDS" not in string]
    # print(idx, new_lst)
    idx += 1
    fywless_tags = pd.concat([fywless_tags, pd.Series([new_lst])], ignore_index=True)
# all_lst_df["CourseTags"] = new_series
# all_lst_df
fywless_tags.columns = ["CourseTags"]
fywless_tags
    # for string in x:
    #     if string.startswith("FYW topics"):

# for x in all_lst_df["CourseTags"]:
#     print(x)
#     new_x = []
#     for item in all_lst_df.loc[x.index, "CourseTags"]:
#         print(item)
#         if not item.startswith("FYW topics"):
#             new_x.append(item)
#     all_lst_df.loc[x, "CourseTags"] = new_x
            # indx -= 1
# all_lst_df[all_lst_df["CourseCode"].apply(lambda x: x and x.startswith("FYW"))]


,CourseTags
0,[empty]
1,[empty]
2,[empty]
3,[empty]
4,[empty]
...,...
3017,[]
3018,"[GER - CORE :: TA (Critical, Analytical Interp..."
3019,[]
3020,[]


In [6]:
# fixing the encoding for "Women's, Gender, and Sexuality Studies"
fywless_tags["CourseTags"] = fywless_tags["CourseTags"].apply(lambda lst: [string.replace("Womenâ€™s", "Women's") for string in lst])
fywless_tags["CourseTags"];

In [7]:
all_lst_df["CourseTags"] = fywless_tags
# now, to find all the GER courses
ger_series = all_lst_df["CourseTags"].apply(lambda lst: " / ".join([string for string in lst if "GER" in string]))
pth_series = all_lst_df["CourseTags"].apply(lambda lst: " / ".join([string for string in lst if "Special Course :: Pathways" in string]))
# all_lst_df[ger_courses.empty]

In [132]:
ger_series;

In [133]:
pth_series;

In [8]:
# NW and NWL considered different categories
ger_pattern = r'HB|IEJ|FYW|WR|NE|WC|HA|TA|VP|UQ|FL|MB|MR|NW\b|NWL'
ger_series = ger_series.apply(lambda string: re.findall(ger_pattern, string) if type(string) == str else string)
# ger_series[ger_series.apply(lambda x: True if re.findall(r'NW[L\s\,]', x) != None else False)]
ger_series


0              []
1              []
2              []
3              []
4              []
          ...    
3017           []
3018    [TA, IEJ]
3019           []
3020           []
3021           []
Name: CourseTags, Length: 3022, dtype: object

In [9]:
# ger_series lsts need to be strings, must be hashable
ger_series = ger_series.apply(lambda x: ", ".join(x) if type(x) == list else x)
pth_series = pth_series.apply(lambda x: ", ".join(x) if type(x) == list else x)
ger_series

0              
1              
2              
3              
4              
         ...   
3017           
3018    TA, IEJ
3019           
3020           
3021           
Name: CourseTags, Length: 3022, dtype: object

In [10]:
ger_cond = ger_series.apply(lambda lst: len(lst) > 0)
pth_df = all_lst_df[pth_series.apply(lambda lst: len(lst) > 0)][["CourseCode", "CourseTitle", "Semester"]]

# ger_df = all_lst_df[ger_cond]
ger_courses = all_lst_df.copy()
ger_courses["CourseTags"] = ger_series
only_gers = ger_courses[ger_cond][["CourseCode", "CourseTitle", "CourseTags", "InstructionalFormat", "Semester"]]

In [33]:
only_gers
only_gers.to_csv("csv-files/gers_raw.csv")

In [182]:
only_gers[only_gers["CourseCode"] == "CHM 115"];

In [11]:
len(only_gers["CourseCode"])

1142

In [12]:
# let's sort by semester to try and get rid of duplicates (keep most recent Prereqs, courseTags)
gers_sorted = only_gers.sort_values(by="Semester", ascending=False).drop_duplicates(subset=["CourseCode", "InstructionalFormat"])
pth_sorted = pth_df.sort_values(by="Semester", ascending=False).drop_duplicates(subset=["CourseCode","CourseTitle"])
# nw + nwl: NW\b|NWL
has_nw = gers_sorted["CourseTags"].apply(lambda x: True if re.findall(r'MR', x) else False)
gers_sorted[has_nw]
# sorted[(sorted["InstructionalFormat"] == "Laboratory") | (sorted["InstructionalFormat"] == "Combination")]

,CourseCode,CourseTitle,CourseTags,InstructionalFormat,Semester
733,CSC 121,Intro to Computer Programming,MR,Laboratory,5_Spring-25
1600,MTH 245,Statistical Methods I with R,MR,Lecture,5_Spring-25
1588,MTH 150,Analytic Geometry and Calculus I,MR,Lecture,5_Spring-25
721,CSC 105,Intro to Computer Science (Creativity and Comp...,MR,Laboratory,5_Spring-25
717,CSC 105,Intro to Computer Science (Creativity and Comp...,MR,Lecture,5_Spring-25
735,CSC 121,Intro to Computer Programming,MR,Lecture,4_Fall-24
1568,MTH 110,Finite Mathematics,MR,Lecture,4_Fall-24
1570,MTH 115,Ideas in Mathematics,MR,Lecture,4_Fall-24
1585,MTH 145,Calculus for Management Life and Social Sciences,MR,Lecture,4_Fall-24
1572,MTH 120,Introduction to Statistics,MR,Lecture,3_Spring-24


In [13]:
pth_sorted = pth_sorted.drop(columns=["Semester"]) if "Semester" in pth_sorted.columns else pth_sorted
pth_sorted = pth_sorted.sort_values(by="CourseCode")
pth_courses = pth_sorted.to_dict(orient="tight")["data"]
pth_courses

[['PTH 101', 'Pathways 101'],
 ['PTH 102', 'Pathways 102'],
 ['PTH 103', 'Pathways 103'],
 ['PTH 201', 'Pathways 201'],
 ['PTH 202', 'Pathways 202'],
 ['PTH 203', 'Pathways 203']]

In [14]:
ger_courses_dict = {"fyw" : [],
                    "pathways" : pth_courses,
                    "wr" : [],                    
                    "ne" : [],
                    "iej" : [],
                    "wc" : [],
                    "ha" : [],
                    "ta" : [],
                    "vp" : [],
                    "uq" : [],
                    "fl" : [],
                    "mb" : [],
                    "mr" : [],
                    "hb" : [],
                    "nw" : [],
                    "nwl" : []
                    }
for ger in list(ger_courses_dict.keys()):
    if ger != "pathways":
        pattern = r'{}'.format(ger.upper())
        if ger == "nw":
            pattern = r'NW\b'
        # print(pattern)
        courses = gers_sorted[gers_sorted["CourseTags"].apply(lambda x: True if re.findall(pattern, x) else False)][["CourseCode", "CourseTitle"]]\
                                .drop_duplicates()\
                                    .sort_values(by="CourseCode")\
                                        .to_dict(orient="tight")["data"]
        ger_courses_dict[ger] = sorted(list(courses))

In [ ]:
# pth_sorted = pth_sorted.drop(columns=["Semester"]) if "Semester" in pth_sorted.columns else pth_sorted
# pth_sorted = pth_sorted.sort_values(by="CourseCode")
# courses = pth_sorted.to_dict(orient="tight")["data"]
# courses

In [39]:
final_json = {}
for ger_cat in ger_courses_dict:
    ger_courses = ger_courses_dict[ger_cat]
    final_json[ger_cat] = [' - '.join(course) for course in ger_courses]

In [42]:
import json 
content = json.dumps(final_json)
f = open('gers.json', 'w')
f.write(content)
f.close()

In [ ]:
# Use this to test dict
# GERS = 'FYW', 'WR', 'NE', 'IEJ', 'WC', 'HA', 'TA', 'VP', 'UQ', 'FL', 'MB', 'MR', 'HB', 'NW', 'NWL'
given_ger = input("Please enter a GER credit: ").lower()
attempts = 0
while given_ger not in ger_courses_dict.keys() and attempts < 3:
    given_ger = input("Please enter a valid GER credit: ").lower()
    attempts += 1
if given_ger in ger_courses_dict.keys():
    print(len(ger_courses_dict[given_ger]))
    for course in ger_courses_dict[given_ger]:
        print(course)
else:
    print("ERROR: {} not in dict".format(given_ger))

77
['FYW 1110', 'Global Water Issues']
['FYW 1111', 'Haunted Mansions']
['FYW 1112', 'History of Liberal Arts']
['FYW 1116', 'Language  Argument and Culture']
['FYW 1126', 'The Ethics of Sex']
['FYW 1127', 'To Walk the Land']
['FYW 1128', 'Turing: Thinking Machines  Codes and Other Enigmas']
['FYW 1133', 'Can We Make Sense of the 60s?']
['FYW 1137', 'Freedom or Oppression: Human Rights in Asia']
['FYW 1140', 'History of Detective Fiction']
['FYW 1141', 'Homer and History']
['FYW 1148', 'Southern Women: Black & White']
['FYW 1151', 'American Dream Ideal and Reality']
['FYW 1154', 'The Wealth of Nations: the Life  Ideas  and influence of Adam Smith']
['FYW 1156', 'Who Speaks Bad English? Language and Ideology']
['FYW 1167', 'American Disaster Literature']
['FYW 1168', 'The First World War']
['FYW 1172', 'Dueling Perspectives: The United States In Latin America']
['FYW 1178', 'Academic Autobiography']
['FYW 1179', 'Our Technological Heritage']
['FYW 1189', 'Social History of Technology']


# Step 9: Create appropriate .csv files for cleaned dfs

In [30]:
ger_courses_dict["nwl"]

[['BIO 101', 'Principles of Biology'],
 ['BIO 102', 'Biology and the Environment'],
 ['BIO 111', 'Foundations of Biology'],
 ['BIO 230', 'Nature of Diversity'],
 ['BIO 423', 'Marine Biology'],
 ['CHM 110', 'Foundations of Chemistry'],
 ['CHM 115', 'Kinetics Thermodynamics and the Environment'],
 ['CHM 118', 'Principles of Chemistry: Theory and Applications'],
 ['CHM 120', 'Organic Chemistry I'],
 ['EES 112', 'Environmental Science'],
 ['EES 113', 'Natural Hazards and Natural Disasters'],
 ['EES 115', 'Earth Systems'],
 ['HSC 311', 'Physiology of Exercise'],
 ['HSC 323', 'Kinesiology'],
 ['PHY 105', 'Descriptive Astronomy'],
 ['PHY 111', 'General Physics I'],
 ['PHY 112', 'General Physics II'],
 ['PSY 320', 'Introduction to Biopsychology'],
 ['PSY 328', 'Sensation and Perception'],
 ['SUS 120', 'Principles of Sustainability Science']]

In [26]:
ger_file_path = "csv-files/gers.csv"

with open(ger_file_path, mode='w', newline='') as f:
    try:
        writer = csv.writer(f)
        writer.writerow(ger_courses_dict.keys())
        writer.writerow(ger_courses_dict.values())
    except Exception as e:
        print(e)
print("Successful!")

Successful!
